## Download datasets from Kaggle


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d ruchi798/bookcrossing-dataset

Dataset URL: https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset
License(s): CC0-1.0
 95% 72.0M/76.1M [00:01<00:00, 55.0MB/s]
100% 76.1M/76.1M [00:01<00:00, 52.0MB/s]


In [ ]:
!unzip bookcrossing-dataset.zip

Archive:  bookcrossing-dataset.zip
replace Book reviews/Book reviews/BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Installation

In [ ]:
pip install dataprep

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 74.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask[array,dataframe,delayed] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This

## Reading CSV

In [ ]:
import pandas as pd
import ast
import numpy as np
import string
from google.colab import drive
import re
from datetime import datetime
drive.mount('/content/drive')
#Users
u_cols = ['id', 'location', 'age']
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/BX-Users.csv', sep=';', names=u_cols, encoding='latin-1',low_memory=False)

#Books
i_cols = ['isbn', 'title' ,'author','year_published', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l']
items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/BX_Books.csv', sep=';', names=i_cols, encoding='latin-1',low_memory=False)

#Ratings
r_cols = ['user_id', 'isbn', 'rating']
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/5500_datasets/Raw_datasets/BX-Book-Ratings.csv', sep=';', names=r_cols, encoding='latin-1',low_memory=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Clean BX-users

### Cleaning

In [ ]:
# Replace ages over 115 with NaN
users['age'] = pd.to_numeric(users['age'], errors='coerce')
users.loc[users['age'] > 100, 'age'] = np.nan
users['age'] = users['age'].astype(pd.Int64Dtype())
# print(users['age'].unique())

In [ ]:
# Split the 'location' column into city, state, and country
location_split = users['location'].str.split(',', expand=True)
users['city'] = location_split[0].str.strip()
users['state'] = location_split[1].str.strip()
users['country'] = location_split[2].str.strip()

In [ ]:
# Only selecting non header rows
users = users.iloc[1:].reset_index(drop=True)
# Countries before cleaning
unique_countries = users['country'].unique().tolist()

print(len(unique_countries)) #1087
print(len(users)) #278858

1087
278858


In [ ]:
#### MATCH WITH WORLD CITIES ##
# Match country
from dataprep.clean import clean_country
users = clean_country(users,"country",output_format="alpha-2",fuzzy_dist=5)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/core.py:7365: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(
                                                                                                          

Country Cleaning Report:
	274147 values cleaned (98.31%)
	6 values unable to be parsed (0.0%), set to NaN
Result contains 274147 (98.31%) values in the correct format and 4711 null values (1.69%)


In [ ]:
users.head()
# users['country_clean'] = users['country_clean'].str.upper()
# unique_clean_countries = users['country_clean'].unique().tolist()
# print(len(unique_clean_countries)) #1007

,id,location,age,city,state,country,country_clean
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa,US
1,2,"stockton, california, usa",18,stockton,california,usa,US
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia,RU
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal,PT
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom,GB


In [ ]:
# Match city
cities = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/11-24_updated_cities.csv")
cities['city'] = cities['city'].str.lower()
cities['country_iso'] = cities['country_iso'].str.upper()
cities.head()

,id,city,latitude,longitude,country_name,country_iso
0,1392685764,tokyo,35.6897,139.6922,Japan,JP
1,1360771077,jakarta,-6.1750,106.8275,Indonesia,ID
2,1356872604,delhi,28.6100,77.2300,India,IN
3,1156237133,guangzhou,23.1300,113.2600,China,CN
4,1356226629,mumbai,19.0761,72.8775,India,IN


In [ ]:
# Inner join on both 'country_clean' and 'city'
matched_rows = users.merge(
    cities[['id', 'country_iso', 'city']],
    left_on=['country_clean', 'city'],
    right_on=['country_iso', 'city'],
    how='left')
matched_rows.head()

,id_x,location,age,city,state,country,country_clean,id_y,country_iso
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa,US,NaN,NaN
1,2,"stockton, california, usa",18,stockton,california,usa,US,1.840022e+09,US
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia,RU,1.643318e+09,RU
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal,PT,1.620357e+09,PT
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom,GB,1.826384e+09,GB


In [ ]:
# Drop duplicates
unique_matches = matched_rows.drop_duplicates(subset=['id_x'])

# Count the number of unique matches - 246086
count_unique_matches = unique_matches.shape[0]
count_unique_matches

278858

In [ ]:
non_nan_count = unique_matches['id_y'].notna().sum()
print("Number of non-NaN rows in 'id_y':", non_nan_count) #124261

Number of non-NaN rows in 'id_y': 210516


In [ ]:
print(users.shape) #(278858, 7)
print(len(users['id'])) #278858
print(unique_matches.shape) #(278858, 10)
print(count_unique_matches)
unique_matches.head(20)

(278858, 7)
278858
(278858, 9)
278858


,id_x,location,age,city,state,country,country_clean,id_y,country_iso
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa,US,NaN,NaN
1,2,"stockton, california, usa",18,stockton,california,usa,US,1.840022e+09,US
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia,RU,1.643318e+09,RU
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal,PT,1.620357e+09,PT
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom,GB,1.826384e+09,GB
6,6,"santa monica, california, usa",61,santa monica,california,usa,US,1.840022e+09,US
7,7,"washington, dc, usa",<NA>,washington,dc,usa,US,1.840006e+09,US
18,8,"timmins, ontario, canada",<NA>,timmins,ontario,canada,CA,1.124761e+09,CA
19,9,"germantown, tennessee, usa",<NA>,germantown,tennessee,usa,US,1.840006e+09,US
22,10,"albacete, wisconsin, spain",26,albacete,wisconsin,spain,ES,1.724000e+09,ES


In [ ]:
unique_matches = unique_matches.rename(columns={'id_x': 'id'})
unique_matches = unique_matches.rename(columns={'id_y': 'location_id'})

In [ ]:
# Enriching for users with no matches
unmatched_users = unique_matches[unique_matches['location_id'].isna()]

# Step 3: For each unmatched user, randomly assign a city id from cities in the same country
def assign_random_city_id(row):
    # Filter cities to get those that match the user's country
    country_cities = cities[cities['country_iso'] == row['country_clean']]

    # If there are cities available for the country, pick a random city ID
    if not country_cities.empty:
        return country_cities['id'].sample(n=1).iloc[0]
    else:
        return np.nan  # In case there are no cities for the country in the cities DataFrame

# Apply the function only to rows with NaN city IDs
unique_matches_1 = unique_matches.copy()
unique_matches_1.loc[unique_matches_1['location_id'].isna(), 'location_id'] = unmatched_users.apply(assign_random_city_id, axis=1)

unique_matches_1['location_id'] = unique_matches_1['location_id'].astype('Int64')
unique_matches_1.head(10)

non_nan_count_1 = unique_matches['location_id'].notna().sum()
print("Number of non-NaN rows in 'location_id':", non_nan_count_1) #124261

Number of non-NaN rows in 'location_id': 210516


In [ ]:
unique_matches['location_id'] = unique_matches['location_id'].astype('Int64')

unique_matches.head(10)

# manually matching NYC to 1840034016
unique_matches.loc[unique_matches['city'] == 'nyc', 'location_id'] = 1840034016

### Output to csv

In [ ]:
selected_columns = ['id', 'location_id', 'age']
df_selected = unique_matches[selected_columns].rename(columns={
    'location_id': 'location_id',
    'id': 'id'
})
df_selected['location_id'] = df_selected['location_id'].astype('Int64').astype(str)
df_selected['age'] = df_selected['age'].astype('Int64').astype(str)

# Display the result
df_selected.head()

,id,location_id,age
0,1,1840034016,<NA>
1,2,1840021517,18
2,3,1643318494,<NA>
3,4,1620356810,17
4,5,1826383788,<NA>


In [ ]:
df_selected = df_selected.where(pd.notnull(df_selected), None)
df_selected['age'] = pd.to_numeric(df_selected['age'], errors='coerce')
df_selected['age'] = df_selected['age'].astype('Int64')
df_selected = df_selected.drop_duplicates(subset=['id'], keep='first')
df_selected['location_id'] = df_selected['location_id'].where(df_selected['location_id'].notna(), None)
df_selected.to_csv('/content/drive/MyDrive/Colab Notebooks/5500_datasets/for_database_upload/bx_users_new_location.csv', index=False)

In [ ]:
# users orginal shape: (278859, 3)
print(cities.shape)
new_df = pd.read_csv('/content/drive/MyDrive/5500_datasets/bx_users.csv', dtype={'location_id': str, 'age': str}, low_memory=False)
print(new_df.shape)
new_df.head(10)

(47868, 6)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/5500_datasets/bx_users.csv'

## Clean BX-Book-Ratings

### Clean

In [ ]:
ratings = ratings.iloc[1:].reset_index(drop=True)
ratings.head(10)


,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0
6,276736,3257224281,8
7,276737,0600570967,6
8,276744,038550120X,7
9,276745,342310538,10


In [ ]:
ratings_unique = ratings['rating'].unique()
print(ratings_unique.tolist())
# Check if all user_ids in `rating` are in users['id']
all_user_ids_exist = ratings['user_id'].isin(users['id']).all()

# Print the result
if all_user_ids_exist:
    print("All user_ids in `rating` are present in `users['id']`.")
else:
    print("Some user_ids in `rating` are not present in `users['id']`.")

['0', '5', '3', '6', '8', '7', '10', '9', '4', '1', '2']
All user_ids in `rating` are present in `users['id']`.


### Output to csv

In [ ]:
max_isbn_length = ratings[ratings['isbn'].apply(lambda x: isinstance(x, str))]['isbn'].str.len().max()
print("Maximum length of ISBNs in `ratings` where values are strings:", max_isbn_length)
print(ratings.shape) # (1149780, 3)

Maximum length of ISBNs in `ratings` where values are strings: 14
(1149780, 3)


In [ ]:
selected_columns = ['user_id', 'isbn', 'rating']  # replace w city id

# Select and rename the columns
df_selected = ratings[selected_columns]

df_selected.to_csv('/content/drive/MyDrive/5500_datasets/bx_reviews.csv', index=False)

In [ ]:
exported_ratings = pd.read_csv('/content/drive/MyDrive/5500_datasets/bx_reviews.csv', low_memory=False)

In [ ]:
print(exported_ratings.shape)

(1149780, 3)


## Clean BX_Books

In [ ]:
#Processed
p_names = ['idx', 'user_id', 'location', 'age', 'isbn', 'rating',
       'book_title', 'book_author', 'year_of_publication', 'publisher',
       'img_s', 'img_m', 'img_l', 'summary', 'language', 'category', 'city',
       'state', 'country']
processed = pd.read_csv('/content/drive/MyDrive/5500_datasets/Raw_datasets/Preprocessed_data.csv', names=p_names,encoding='latin-1',low_memory=False)

In [ ]:
selected_columns = processed[['isbn','summary', 'language', 'category']]
selected_unique = selected_columns.drop_duplicates()

In [ ]:
items['year_published'] = pd.to_numeric(items['year_published'], errors='coerce')
items['year_published'] = items['year_published'].apply(lambda x: np.nan if x == 0 or x > 2004 else x)
items['year_published'] = items['year_published'].astype('Int64')

unique_year = items['year_published'].unique()
print(items['year_published'].max()) # 2004

2004


In [ ]:
print(items.shape) #(271380, 8)

(271380, 8)


In [ ]:
items['isbn'] = items['isbn'].astype(str)
selected_unique['isbn'] = selected_unique['isbn'].astype(str)

# Perform the merge on 'isbn'
merged_items = items.merge(selected_unique, on='isbn', how='left')
print(merged_items.head())

<ipython-input-8-e9a77ea0d922>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_unique['isbn'] = selected_unique['isbn'].astype(str)


         isbn                                              title  \
0        ISBN                                         Book-Title   
1  0195153448                                Classical Mythology   
2  0002005018                                       Clara Callan   
3  0060973129                               Decision in Normandy   
4  0374157065  Flu: The Story of the Great Influenza Pandemic...   

                 author  year_published                publisher  \
0           Book-Author            <NA>                Publisher   
1    Mark P. O. Morford            2002  Oxford University Press   
2  Richard Bruce Wright            2001    HarperFlamingo Canada   
3          Carlo D'Este            1991          HarperPerennial   
4      Gina Bari Kolata            1999     Farrar Straus Giroux   

                                         image_url_s  \
0                                        Image-URL-S   
1  http://images.amazon.com/images/P/0195153448.0...   
2  http://imag

In [ ]:
merged_items['category'] = merged_items['category'].apply(lambda x: np.nan if '9' in str(x) else x)
merged_items['category'] = merged_items['category'].str.replace(r"[\[\]']", "", regex=True)
merged_items['category'] = merged_items['category'].str.lower()

merged_items_unique = merged_items['category'].unique().tolist()

print(merged_items_unique)
print(len(merged_items_unique)) # 6303

[nan, 'social science', 'actresses', 'medical', 'design', 'fiction', 'history', 'nature', 'humor', 'cooking', 'reference', 'juvenile fiction', 'canadian fiction', 'health & fitness', 'control (psychology)', 'cholistan desert (pakistan)', 'science', 'political ethics', 'conduct of life', 'horses', 'political science', 'brothers', 'baggins, frodo (fictitious character)', 'biography & autobiography', 'business & economics', 'americans', 'infanticide', 'cabala', 'literary criticism', 'self-help', 'american fiction', 'humorous stories', 'diary fiction', 'religion', 'german language', 'bloom, leopold (fictitious character)', 'detective and mystery stories', 'gargoyles (fictitious characters)', 'oglala indians', 'england', 'comic books, strips, etc', 'comics & graphic novels', 'language arts & disciplines', 'aerospace engineers', 'greek poetry', 'juvenile nonfiction', 'gay men', 'orphans', 'family problems', 'computers', 'american literature', 'popular music', 'businessmen', 'psychology', 'bo

In [ ]:
books_merged = merged_items.iloc[1:].reset_index(drop=True)

In [ ]:
books_merged.head()
max_lengths = books_merged.apply(lambda col: col.astype(str).str.len().max())

# Display the result
print(max_lengths)
print(books_merged.shape)

isbn               13
title             256
author            143
year_published      4
publisher         134
image_url_s        60
image_url_m        60
image_url_l        60
summary           374
language            5
category          276
dtype: int64
(271379, 11)


### Export

In [ ]:
books_merged['film_name'] = np.nan
books_merged['classification'] = np.nan
books_merged['set_in_location'] = np.nan
books_merged['time_period_start'] = np.nan
books_merged['time_period_end'] = np.nan

books_merged.to_csv('/content/drive/MyDrive/5500_datasets/bx_books.csv', index=False)

In [ ]:
books_cleaned = pd.read_csv('/content/drive/MyDrive/5500_datasets/bx_books.csv', low_memory=False)
books_cleaned.head()

,isbn,title,author,year_published,publisher,image_url_s,image_url_m,image_url_l,summary,language,category,film_name,classification,set_in_location,time_period_start,time_period_end
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,social science,NaN,NaN,NaN,NaN,NaN
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,actresses,NaN,NaN,NaN,NaN,NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,"Here, for the first time in paperback, is an o...",en,NaN,NaN,NaN,NaN,NaN,NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,"Describes the great flu epidemic of 1918, an o...",en,medical,NaN,NaN,NaN,NaN,NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,A look at the incredibly well-preserved ancien...,en,design,NaN,NaN,NaN,NaN,NaN


### Analysis of BX/Amazon Overlap

In [ ]:
amazon_books = pd.read_csv('/content/drive/MyDrive/5500_datasets/books_w_id.csv', low_memory=False)

In [ ]:
###############################
# Overlapping # of categories #
###############################
amazon_cat = amazon_books['categories'].str.replace(r"[\[\]']", "", regex=True).str.lower()
amazon_cat_unique = pd.DataFrame(amazon_cat.unique(), columns=['categories'])

bx_cat_unique = pd.DataFrame(books_merged['category'].unique(), columns=['category'])
merged_cats = amazon_cat_unique.merge(bx_cat_unique, left_on='categories', right_on='category', how='inner')

# Print the result
print(merged_cats.head(30))
print("Number of merged categories:", len(merged_cats)) # 3419
print("Distinct number of categories - Amazon: ", amazon_cat_unique.shape[0]) # 10808
print("Distinct number of categories - BX: ", bx_cat_unique.shape[0]) # 6303

                     categories                     category
0       comics & graphic novels      comics & graphic novels
1     biography & autobiography    biography & autobiography
2                      religion                     religion
3                       fiction                      fiction
4                           NaN                          NaN
5                social science               social science
6                     reference                    reference
7           juvenile nonfiction          juvenile nonfiction
8      technology & engineering     technology & engineering
9                       history                      history
10          new zealand fiction          new zealand fiction
11                          law                          law
12            political science            political science
13             health & fitness             health & fitness
14                      cooking                      cooking
15                   phi

In [ ]:
###############################
##### Overlapping ISBNs #######
###############################

# Amazon (212404, 12)
# BX (271380, 8)
# Overlap = 16600

overlapping_isbns = books_merged.merge(amazon_books, left_on='isbn', right_on="Id", how='inner')
num_overlapping = len(overlapping_isbns)
print("Number of overlapping rows based on 'isbn':", num_overlapping)

Number of overlapping rows based on 'isbn': 16600


In [ ]:
################################################
##### Overlapping Rows Based on Category #######
################################################
amazon_books['categories'] = amazon_cat

amazon_matches = amazon_books[amazon_books['categories'].isin(merged_cats['category'])].shape[0]
print("Number of rows in amazon_books with categories present in merged_cats:", amazon_matches) # 200373

bx_matches = books_merged[books_merged['category'].isin(merged_cats['category'])].shape[0]
print("Number of rows in bx_books with categories present in merged_cats:", bx_matches) # 267617



Number of rows in amazon_books with categories present in merged_cats: 200373
Number of rows in bx_books with categories present in merged_cats: 267617


## For bx enrichment

In [ ]:
import pandas as pd
import numpy as np

bx_books = pd.read_csv('/content/drive/MyDrive/5500_datasets/bx_books.csv', low_memory=False)
bx_reviews = pd.read_csv('/content/drive/MyDrive/5500_datasets/bx_reviews.csv', low_memory=False)


In [ ]:
bx_books.head()

,isbn,title,author,year_published,publisher,image_url_s,image_url_m,image_url_l,summary,language,category,film_name,classification,set_in_location,time_period_start,time_period_end
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,social science,NaN,NaN,NaN,NaN,NaN
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,actresses,NaN,NaN,NaN,NaN,NaN
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,"Here, for the first time in paperback, is an o...",en,NaN,NaN,NaN,NaN,NaN,NaN
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,"Describes the great flu epidemic of 1918, an o...",en,medical,NaN,NaN,NaN,NaN,NaN
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,A look at the incredibly well-preserved ancien...,en,design,NaN,NaN,NaN,NaN,NaN


In [ ]:
bx_reviews.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [ ]:
merged = bx_books.merge(bx_reviews, on="isbn", how="left")
merged['rating'] = merged['rating'].fillna(0)
rating_counts = merged.groupby(["isbn"])['rating'].count().reset_index(name="count_of_ratings")

print(bx_books.shape)

rating_counts.head()

(271379, 16)


,isbn,count_of_ratings
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1


In [ ]:
print(rating_counts.shape)

(271379, 2)


In [ ]:
final_merged = rating_counts.merge(bx_books[['isbn','author','title']], on="isbn", how="left")

In [ ]:
print(final_merged.shape)
final_merged.head()

(271379, 4)


,isbn,count_of_ratings,author,title
0,0000913154,1,C. van Amerongen (translator),The Way Things Work: An Illustrated Encycloped...
1,0001010565,2,Judith Kerr,Mog's Christmas
2,0001046438,1,Stephen Fry,Liar
3,0001046713,1,Helen Forrester,Twopence to Cross the Mersey
4,000104687X,1,T.S. Eliot,"T.S. Eliot Reading \The Wasteland\"" and Other ..."


In [ ]:
final_merged.to_csv('/content/drive/MyDrive/5500_datasets/enrichments/bx_books_w_rating_counts.csv', index=False)